In [18]:
import numpy as np
# from sympy.utilities.iterables import multiset_permutations
import random
from itertools import count
from queue import PriorityQueue

tie = count()  # global (serves as a tie breaker for elements with same fitness in the priority Queue )
import time

In [17]:
class Map:
    def __init__(self, input):
        self.given_map = []
        self.input = input
        self.industry = 0
        self.residential = 0
        self.commercial = 0

    def get_map(self):
        with open(self.input) as f:
            # whenever \n is encountered splitlines will break and take the encountered part aas a string
            my_list = f.read().splitlines()

        # my_list = [x.strip() for x in my_list.split(',')]
        self.industry = int(my_list[0])
        self.residential = int(my_list[1])
        self.commercial = int(my_list[2])
        for i in range(3):
            del my_list[0]

        for i in range(len(my_list)):
            self.given_map.append(my_list[i].split(','))

        for i in range(len(self.given_map)):
            for j in range(len(self.given_map[0])):
                if self.given_map[i][j].isdigit():
                    self.given_map[i][j] = int(self.given_map[i][j])
                elif self.given_map[i][j] == 'X':
                    self.given_map[i][j] = 10
                elif self.given_map[i][j] == 'S':
                    self.given_map[i][j] = 11

        # coverting into array
        self.given_map = np.array(self.given_map)


class Environment(Map):
    def __init__(self, input):
        Map.__init__(self, input)
        self.get_map()
        self.score = 0
        self.manhattan_check_non_res = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1],
                                                 [1, 0], [1, 1], [0, -2], [0, 2], [2, 0], [-2, 0]])
        # self.manhattan_check_res = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1],
        #                        [1, 0], [1, 1], [0, -2], [0, 2], [2, 0], [-2, 0]])
        self.manhattan_check_res = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1],
                                             [1, 0], [1, 1], [-2, -2], [-2, 0], [-2, 2], [0, -2], [0, 2], [2, -2],
                                             [2, 0], [2, 2],
                                             [0, -3], [0, 3], [3, 0], [-3, 0]])
        self.i_indices = []
        self.r_indices = []
        self.c_indices = []
        self.a = []

    def get_score(self, my_map):
        self.score = 0
        x_indices = np.asarray(np.where(self.given_map == 10)).T
        s_indices = np.asarray(np.where(self.given_map == 11)).T
        i_indices = np.asarray(np.where(my_map == 12)).T
        r_indices = np.asarray(np.where(my_map == 13)).T
        c_indices = np.asarray(np.where(my_map == 14)).T
        # print(x_indices)
        # print(s_indices)
        # print(i_indices)
        # print(r_indices)
        # print(c_indices)
        for indices in x_indices:
            x_neighbours = self.manhattan_check_non_res + indices
            x_neighbours = (x_neighbours.T[:, x_neighbours.T.min(axis=0) >= 0]).T
            for i, j in x_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 12:
                        # print('1')
                        self.score -= 10
                    if my_map[i, j] == 13 or my_map[i, j] == 14:
                        # print('2')
                        self.score -= 20
                except IndexError:
                    pass

        for indices in s_indices:
            if my_map[indices[0], indices[1]] != 12 and my_map[indices[0], indices[1]] != 13 and my_map[
                indices[0], indices[1]] != 14:
                s_neighbours = self.manhattan_check_non_res + indices
                s_neighbours = (s_neighbours.T[:, s_neighbours.T.min(axis=0) >= 0]).T
                for i, j in s_neighbours:
                    # print(i,j)
                    try:
                        if my_map[i, j] == 13:
                            # print('3')
                            self.score += 10
                    except IndexError:
                        pass

        for indices in i_indices:
            self.a = indices
            i_neighbours = self.manhattan_check_non_res + indices
            i_neighbours = (i_neighbours.T[:, i_neighbours.T.min(axis=0) >= 0]).T
            for i, j in i_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 12:
                        # print('4')
                        self.score += 3
                except IndexError:
                    pass

            if self.given_map[indices[0], indices[1]] != 11:
                # print('5')
                self.score -= self.given_map[indices[0], indices[1]]

        for indices in c_indices:
            # print(indices)
            c_neighbours = self.manhattan_check_non_res + indices
            c_neighbours = (c_neighbours.T[:, c_neighbours.T.min(axis=0) >= 0]).T
            for i, j in c_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 14:
                        # print('6')
                        self.score -= 5

                except IndexError:
                    pass

            if self.given_map[indices[0], indices[1]] != 11:
                # print('5')
                self.score -= self.given_map[indices[0], indices[1]]

        for indices in r_indices:
            r_neighbours = self.manhattan_check_res + indices
            # print(r_neighbours)
            r_neighbours = (r_neighbours.T[:, r_neighbours.T.min(axis=0) >= 0]).T
            for i, j in r_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 12:
                        # print('8')
                        self.score -= 5
                    if my_map[i, j] == 14:
                        # print('9')
                        self.score += 5
                except IndexError:
                    pass

            if self.given_map[indices[0], indices[1]] != 11:
                # print('5')
                self.score -= self.given_map[indices[0], indices[1]]

In [19]:
class Genetic_Algortithm(Environment):
    def __init__(self, input):
        Environment.__init__(self, input)
        self.size = 100
        self.my_map = []
        self.population = []
        self.min_sorted_population = PriorityQueue(0)
        self.max_sorted_population = PriorityQueue(0)
        self.elite = []
        self.offspring = []
        self.culled = []
        self.culled_index = np.zeros(100)
        self.population_proper = []
        self.time = 0
        self.temp_max = 0
        self.temp_best = []
        self.max_score = float("inf")
        self.best_map = []
        self.scores = []
        self.best_time = 0

    def initialize_map(self):
        self.my_map = np.zeros(self.given_map.shape).flatten()
        self.my_map[0:self.industry] = 12
        self.my_map[self.industry:self.residential + self.industry] = 13
        self.my_map[self.residential + self.industry:self.residential + self.industry + self.commercial] = 14
        random.shuffle(self.my_map)

    def populate(self):
        i = 0
        while True:
            self.temp_map = self.my_map.copy()
            random.shuffle(self.temp_map)
            a = self.temp_map.reshape(self.given_map.shape)
            b_1 = a == 12
            b_2 = a == 13
            b_3 = a == 14
            if 10 not in self.given_map[b_1] and 10 not in self.given_map[b_2] and 10 not in self.given_map[b_3]:
                self.population.append(a)
                i += 1
            if i == self.size:
                break

    '''def populate(self):
        i=0
        for pops in multiset_permutations(self.my_map):
            a = np.array(pops).reshape(self.given_map.shape)
            b_1 = a==12
            b_2 = a==13
            b_3 = a==14
            if 10 not in self.given_map[b_1] and 10 not in self.given_map[b_2] and 10 not in self.given_map[b_3]:
                self.permutations.append(a)

        a = random.sample(range(0,len(self.permutations)),self.size)
        for i in a:
            self.population.append(self.permutations[i])'''

    def sort_population(self):
        self.min_sorted_population = PriorityQueue(0)
        self.max_sorted_population = PriorityQueue(0)
        for my_map in self.population:
            self.get_score(my_map)
            self.min_sorted_population.put([self.score, next(tie), my_map])
            self.max_sorted_population.put([-self.score, next(tie), my_map])

    '''def repopulate(self):
        self.population_proper = []
        self.culled_index = np.zeros(self.size)
        for i in self.culled:
            self.culled_index = self.culled_index + np.array([int(np.array_equal(i,x)) for x in self.population])
            print(sum(self.culled_index))
        for i in range(len(self.culled_index)):
            if self.culled_index[i]==0:
                self.population_proper.append(self.population[i])'''

    def repopulate(self):
        self.population_proper = []
        for i in range(self.size - len(self.culled)):
            a = self.min_sorted_population.get()
            self.population_proper.append(a[2])

    def culling(self):
        self.culled = []
        for i in range(int(0.1 * self.size)):
            b = self.min_sorted_population.get()
            self.culled.append(b[2])

    def elitism(self):
        self.elite = []
        for i in range(int(0.1 * self.size)):
            a = self.max_sorted_population.get()
            if i == 0:
                self.temp_max = a[0]
                self.temp_best = a[2].copy()
            self.elite.append(a[2])

    def crossover(self):
        '''
        Perform mating and produce new offspring
        '''
        self.offspring = []
        numbers = 0
        while True:
            length = len(self.culled)
            int1 = np.random.randint(0, self.size - length)
            int2 = np.random.randint(0, self.size - length)
            if int1 != int2:
                par_1 = self.population_proper[int1]
                par_2 = self.population_proper[int2]

                # random probability
                prob = random.random()

                # if prob is less than 0.35, change industrial tiles
                if prob < 0.35:
                    b_1 = par_1 == 12
                    b_2 = par_2 == 12
                    if 13 not in par_1[b_2] and 14 not in par_2[b_1] and 13 not in par_2[b_1] and 14 not in par_1[b_2]:
                        par_1[b_1] = 0
                        par_2[b_2] = 0
                        par_1[b_2] = 12
                        par_2[b_1] = 12
                        self.offspring.append(par_1)
                        self.offspring.append(par_2)
                        numbers += 2

                # if prob is between 0.35 and 0.70, change residential tiles
                elif prob >= 0.35 and prob < 0.7:
                    b_1 = par_1 == 13
                    b_2 = par_2 == 13
                    if 12 not in par_1[b_2] and 14 not in par_2[b_1] and 12 not in par_2[b_1] and 14 not in par_1[b_2]:
                        par_1[b_1] = 0
                        par_2[b_2] = 0
                        par_1[b_2] = 13
                        par_2[b_1] = 13
                        self.offspring.append(par_1)
                        self.offspring.append(par_2)
                        numbers += 2

                elif prob >= 0.7 and prob <= 1.0:
                    b_1 = par_1 == 14
                    b_2 = par_2 == 14
                    if 12 not in par_1[b_2] and 13 not in par_2[b_1] and 12 not in par_2[b_1] and 13 not in par_1[b_2]:
                        par_1[b_1] = 0
                        par_2[b_2] = 0
                        par_1[b_2] = 14
                        par_2[b_1] = 14
                        self.offspring.append(par_1)
                        self.offspring.append(par_2)
                        numbers += 2

            if numbers == self.size - len(self.elite):
                break

    def mutate(self):
        s = 0
        while s < 20:
            i = np.random.randint(0, len(self.population_proper))
            mutant = self.population_proper[i].copy()
            while True:
                i1 = np.random.randint(0, self.given_map.shape[0])
                i2 = np.random.randint(0, self.given_map.shape[0])
                j1 = np.random.randint(0, self.given_map.shape[1])
                j2 = np.random.randint(0, self.given_map.shape[1])
                if self.given_map[i1, j1] != 10 and self.given_map[i2, j2] != 10:
                    break
            if mutant[i1, j1] != 0 or mutant[i2, j2] != 0:
                a = mutant[i1, j1]
                mutant[i1, j1] = mutant[i2, j2]
                mutant[i2, j2] = a
                self.population_proper[i] = mutant.copy()
                s += 1

    def evolve(self):
        self.initialize_map()
        self.populate()
        self.time = time.time() + 10
        i = 1
        while True:
            print("Generation", i + 1)
            self.sort_population()
            self.culling()
            # print(len(self.culled))
            self.elitism()
            if self.temp_max < self.max_score:
                self.max_score = self.temp_max
                self.best_map = self.temp_best.copy()
                self.best_time = 10 - (self.time - time.time())
            self.scores.append(-self.max_score)
            print("Max score of population is", -self.max_score)
            # print(len(self.elite))
            self.repopulate()
            self.mutate()
            # print(len(self.population_proper))
            self.crossover()
            # print(len(self.offspring))
            self.offspring.extend(self.elite)
            self.population = self.offspring[:]
            # print(len(self.population))
            i += 1
            if time.time() > self.time:
                break


In [21]:
ga = Genetic_Algortithm('urban 1.txt')
ga.initialize_map()
ga.populate()
ga.sort_population()

In [22]:
print(ga.population)

[array([[ 0.,  0., 13.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0., 14., 12.]]), array([[ 0.,  0., 14.,  0.],
       [ 0., 13.,  0.,  0.],
       [12.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0., 14.],
       [ 0.,  0., 12.,  0.],
       [13.,  0.,  0.,  0.]]), array([[ 0.,  0., 13., 14.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 12.]]), array([[ 0.,  0.,  0.,  0.],
       [12., 13.,  0., 14.],
       [ 0.,  0.,  0.,  0.]]), array([[ 0.,  0.,  0., 14.],
       [ 0.,  0., 13.,  0.],
       [ 0.,  0.,  0., 12.]]), array([[ 0., 13.,  0., 14.],
       [ 0.,  0.,  0.,  0.],
       [12.,  0.,  0.,  0.]]), array([[ 0.,  0., 13., 12.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 14.]]), array([[ 0.,  0.,  0., 12.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0., 13., 14.]]), array([[ 0.,  0., 12.,  0.],
       [14.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 13.]]), array([[ 0.,  0.,  0.,  0.],
       [ 0., 13., 14.,  0.],
       [ 0., 12.,  0.,  0.]]), array([[ 0., 13., 12

In [26]:
# ga.min_sorted_population.queue

In [43]:
a1 = ga.population[0]
a2 = ga.population[1]

In [46]:
a1[0,0] = 12

In [47]:
print(a1)
print(a2)

[[12.  0. 13.  0.]
 [ 0.  0.  0.  0.]
 [12.  0. 14.  0.]]
[[ 0.  0. 14.  0.]
 [ 0. 13.  0.  0.]
 [ 0.  0.  0. 12.]]


In [48]:
b1 = a1==12
b2 = a2 == 12
print(b1)
print(b2)

[[ True False False False]
 [False False False False]
 [ True False False False]]
[[False False False False]
 [False False False False]
 [False False False  True]]


In [49]:
if 13 not in a1[b2] and 14 not in a1[b2] and 13 not in a2[b1] and 14 not in a2[b1]:
    a1[b1]=0
    a2[b2]=0
    a1[b2]=12
    a2[b1]=12

In [50]:
a1

array([[ 0.,  0., 13.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0., 14., 12.]])

In [51]:
a2

array([[12.,  0., 14.,  0.],
       [ 0., 13.,  0.,  0.],
       [12.,  0.,  0.,  0.]])

In [1]:
from random import choice
from collections import Counter
from random import randrange

In [62]:
import numpy as np
#from sympy.utilities.iterables import multiset_permutations
import random
from itertools import count
from queue import PriorityQueue
tie = count() #global
import time

In [125]:
class Map:
    def __init__(self, input):
        self.given_map = []
        self.input = input
        self.industry = 0
        self.residential = 0
        self.commercial = 0

    def get_map(self):
        with open(self.input) as f:
            # whenever \n is encountered splitlines will break and take the encountered part aas a string
            my_list = f.read().splitlines()

        # my_list = [x.strip() for x in my_list.split(',')]
        self.industry = int(my_list[0])
        self.residential = int(my_list[1])
        self.commercial = int(my_list[2])
        for i in range(3):
            del my_list[0]

        for i in range(len(my_list)):
            self.given_map.append(my_list[i].split(','))

        for i in range(len(self.given_map)):
            for j in range(len(self.given_map[0])):
                if self.given_map[i][j].isdigit():
                    self.given_map[i][j] = int(self.given_map[i][j])
                elif self.given_map[i][j] == 'X':
                    self.given_map[i][j] = 10
                elif self.given_map[i][j] == 'S':
                    self.given_map[i][j] = 11

        # coverting into array
        self.given_map = np.array(self.given_map)

In [173]:
class Environment(Map):
    def __init__(self, input):
        Map.__init__(self, input)
        self.get_map()
        self.score = 0
        self.manhattan_check_non_res = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1],
                                                 [1, 0], [1, 1], [0, -2], [0, 2], [2, 0], [-2, 0]])
        # self.manhattan_check_res = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1],
        #                        [1, 0], [1, 1], [0, -2], [0, 2], [2, 0], [-2, 0]])
        self.manhattan_check_res = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1],
                                             [1, 0], [1, 1], [-2, -2], [-2, 0], [-2, 2], [0, -2], [0, 2], [2, -2],
                                             [2, 0], [2, 2],
                                             [0, -3], [0, 3], [3, 0], [-3, 0]])
        self.i_indices = []
        self.r_indices = []
        self.c_indices = []
        self.a = []
        self.temp_map = []

    def get_actual_positions(self, my_map):
        self.i_indices = np.asarray(np.where(my_map == 12)).T
        self.r_indices = np.asarray(np.where(my_map == 13)).T
        self.c_indices = np.asarray(np.where(my_map == 14)).T

    def get_score(self, my_map):
        self.score = 0
        x_indices = np.asarray(np.where(self.given_map == 10)).T
        s_indices = np.asarray(np.where(self.given_map == 11)).T
        i_indices = np.asarray(np.where(my_map == 12)).T
        r_indices = np.asarray(np.where(my_map == 13)).T
        c_indices = np.asarray(np.where(my_map == 14)).T
        # print(x_indices)
        # print(s_indices)
        # print(i_indices)
        # print(r_indices)
        # print(c_indices)
        for indices in x_indices:
            x_neighbours = self.manhattan_check_non_res + indices
            x_neighbours = (x_neighbours.T[:, x_neighbours.T.min(axis=0) >= 0]).T
            for i, j in x_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 12:
                        # print('1')
                        self.score -= 10
                    if my_map[i, j] == 13 or my_map[i, j] == 14:
                        # print('2')
                        self.score -= 20
                except IndexError:
                    pass

        for indices in s_indices:
            if my_map[indices[0], indices[1]] != 12 and my_map[indices[0], indices[1]] != 13 and my_map[
                indices[0], indices[1]] != 14:
                s_neighbours = self.manhattan_check_non_res + indices
                s_neighbours = (s_neighbours.T[:, s_neighbours.T.min(axis=0) >= 0]).T
                for i, j in s_neighbours:
                    # print(i,j)
                    try:
                        if my_map[i, j] == 13:
                            # print('3')
                            self.score += 10
                    except IndexError:
                        pass

        for indices in i_indices:
            self.a = indices
            i_neighbours = self.manhattan_check_non_res + indices
            i_neighbours = (i_neighbours.T[:, i_neighbours.T.min(axis=0) >= 0]).T
            for i, j in i_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 12:
                        # print('4')
                        self.score += 3
                except IndexError:
                    pass

            if self.given_map[indices[0], indices[1]] != 11:
                # print('5')
                self.score -= self.given_map[indices[0], indices[1]]

        for indices in c_indices:
            # print(indices)
            c_neighbours = self.manhattan_check_non_res + indices
            c_neighbours = (c_neighbours.T[:, c_neighbours.T.min(axis=0) >= 0]).T
            for i, j in c_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 14:
                        # print('6')
                        self.score -= 5

                except IndexError:
                    pass

            if self.given_map[indices[0], indices[1]] != 11:
                # print('5')
                self.score -= self.given_map[indices[0], indices[1]]

        for indices in r_indices:
            r_neighbours = self.manhattan_check_res + indices
            # print(r_neighbours)
            r_neighbours = (r_neighbours.T[:, r_neighbours.T.min(axis=0) >= 0]).T
            for i, j in r_neighbours:
                # print(i,j)
                try:
                    if my_map[i, j] == 12:
                        # print('8')
                        self.score -= 5
                    if my_map[i, j] == 14:
                        # print('9')
                        self.score += 5
                except IndexError:
                    pass

            if self.given_map[indices[0], indices[1]] != 11:
                # print('5')
                self.score -= self.given_map[indices[0], indices[1]]

    def move_tile_simulate(self, indices, column, row, type):
        a = self.my_map[row, column]
        if self.given_map[row, column] != 10 and a != 12 and a != 13 and a != 14:
            self.temp_map = self.my_map.copy()
            self.temp_map[indices[0], indices[1]] = 0
            self.temp_map[row, column] = type
            return True
        else:
            return False

    def move_tile_actual(self, indices, column, row, type):
        self.my_map[indices[0], indices[1]] = 0
        self.my_map[row, column] = type

In [185]:
class Hill_Climbing(Environment):
    def __init__(self, input):
        Environment.__init__(self, input)
        self.size = 100
        self.my_map = []
        self.time = 0
        self.max_score = 0
        self.solution = []
        self.solution_score = 0
        self.time = 0
        self.time_best = 0

    def initialize_map(self):
        # get a map with random positions 
        self.my_map = np.zeros(self.given_map.shape).flatten()
        self.my_map[0:self.industry] = 12
        self.my_map[self.industry:self.residential + self.industry] = 13
        self.my_map[self.residential + self.industry:self.residential + self.industry + self.commercial] = 14
        random.shuffle(self.my_map)
        self.my_map = self.my_map.reshape(self.given_map.shape)
        self.get_score(self.my_map)
        # print(self.score)
        self.max_score = self.score
        self.decision = []
        self.scores = []

    def Climb_hill(self):
        rows = self.my_map.shape[0]
        columns = self.my_map.shape[1]
        i = 0
        self.time = time.time() + 10
        while True:

            while True:
                self.get_actual_positions(self.my_map)
                self.decision = []
                for row in range(rows):
                    for column in range(columns):
                        for indices in self.i_indices:
                            if self.move_tile_simulate(indices, column, row, 12):
                                self.get_score(self.temp_map)
                                # print(self.score)

                                if self.score > self.max_score:
                                    self.max_score = self.score
                                    self.decision = [indices, column, row, 12]

                        for indices in self.r_indices:
                            if self.move_tile_simulate(indices, column, row, 13):
                                self.get_score(self.temp_map)
                                if self.score > self.max_score:
                                    self.max_score = self.score
                                    self.decision = [indices, column, row, 13]

                        for indices in self.c_indices:
                            if self.move_tile_simulate(indices, column, row, 14):
                                self.get_score(self.temp_map)
                                if self.score > self.max_score:
                                    self.max_score = self.score
                                    self.decision = [indices, column, row, 14]
                # print(self.max_score)
                if self.decision:
                    self.move_tile_actual(self.decision[0], self.decision[1], self.decision[2], self.decision[3])
                    self.get_score(self.my_map)
                    # print(self.decision)
                    # print(self.my_map)
                    if self.score > self.solution_score:
                        self.solution_score = self.score
                        self.solution = [self.score, self.my_map]
                        self.time_best = 10 - (self.time - time.time())
                if not self.decision:
                    self.initialize_map()
                    print("Restart", i + 1)
                    i += 1
                    break

            if time.time() > self.time:
                break

In [186]:
city = Hill_Climbing('urban 1.txt')

In [198]:
city.initialize_map()

In [199]:
city.my_map

array([[ 0.,  0.,  0., 14.],
       [ 0.,  0., 13.,  0.],
       [ 0., 12.,  0.,  0.]])

In [200]:
city.given_map

array([[10,  1,  2,  4],
       [ 3,  4, 11,  3],
       [ 6,  0,  2,  3]])

In [201]:
city.get_score(city.my_map)
city.score

-4

In [202]:
city.get_actual_positions(city.my_map)
print(city.i_indices)
print(city.r_indices)
print(city.c_indices)

[[2 1]]
[[1 2]]
[[0 3]]


In [208]:
# city.manhattan_check_non_res

In [204]:
i_indices = city.i_indices
print(i_indices)

[[2 1]]


In [205]:
rows = city.my_map.shape[0]
columns = city.my_map.shape[1]
print(rows,columns)

3 4


In [206]:
 for row in range(rows):
    for column in range(columns):
        for indices in i_indices:
            if city.move_tile_simulate(indices, column, row, 12):
                city.get_score(city.temp_map)

In [211]:
city.move_tile_simulate(i_indices[0],0,0,12)

False

In [212]:
city.move_tile_actual(i_indices[0],0,0,12)

In [214]:
city.my_map

array([[12.,  0.,  0., 14.],
       [ 0.,  0., 13.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [2]:
class SearchProblem:
    # Inicia a busca(recebe os parametros iniciais)
    def __init__(self, initial=None):
        pass

    # Define o estado inicial
    def initial(self):
        pass

    # Teste de objetivo
    def goal_test(self, state):
        pass

    # Heuristica, utilizada para problemas de maximizacao ou minimizacao
    def heuristic(self, state):
        pass

    # Retorna os estados acessiveis a partir do estado atual
    def nearStates(self, state):
        pass

    # Retorna uma escolha aleatoria dentre os estados proximos
    def randomNearState(self, state):
        return choice(self.nearStates(state))

In [3]:
class NQueensSearch(SearchProblem):

    def __init__(self, N):
        self.N = N

    def initial(self):
        return list(randrange(self.N) for i in range(self.N))

    # Teste de objetivo:
    #   Tests if any row / column / diagonal is populated by more than one queen
    def goal_test(self, state):
        a, b, c = (set() for i in range(3))
        for row, col in enumerate(state):
            if col in a or row - col in b or row + col in c:
                return False
            a.add(col)
            b.add(row - col)
            c.add(row + col)
        return True

    # Heuristica: h
    #   Number of pairs of queens attacking each other
    def heuristic(self, state):
        # define a,b,c como contadores
        a, b, c = [Counter() for i in range(3)]
        # contar quantas rainhas tem o os valores (a,b,c)
        # de forma que se obtem por exemplo quantas rainhas tem o valor de a=1
        for row, col in enumerate(state):
            a[col] += 1
            b[row - col] += 1
            c[row + col] += 1
        h = 0  # start collisions with 0
        # scans the counting structures (a, b, c) just increasing the collision value
        # case for some value of (a / b / c)> 1 since cnt is done [key] -1
        # divides to remove double counts
        for count in [a, b, c]:
            for key in count:
                h += count[key] * (count[key] - 1) / 2
        return -h

    # Children ou estados vizinhos: children[]
    #   Retorna todos os estados acessiveis a partir do atual movendo as pecas por coluna
    def nearStates(self, state):
        near_states = []
        # Para cada state[coluna] verfica se as colunas vizinhas estao vazias
        for row in range(self.N):
            for col in range(self.N):
                # Se for diferente:
                #   entao a col atual da iteracao esta disponivel para movimentar-se
                #   visto que o state[] guarda o valor das colunas em que estao as rainhas
                if col != state[row]:
                    aux = list(state)
                    aux[row] = col  # Troca a coluna para a vazia
                    near_states.append(list(aux))  # E inclui na lista de nearStates
        return near_states

In [14]:
state = list(randrange(4) for i in range(4))
state

[3, 0, 0, 0]

In [15]:
def goal_test(state):
    a, b, c = (set() for i in range(3))
    print(a,b,c)
    for row, col in enumerate(state):
        print(row,col)
        if col in a or row - col in b or row + col in c:
            return False
        a.add(col)
        b.add(row - col)
        c.add(row + col)
    return True


In [16]:
goal_test(state)

set() set() set()
0 3
1 0
2 0


False

In [3]:
import random
import queue
import timeit

In [4]:

def random_list(start, end, number):
    start = int(start)
    end = int(end)
    number = int(abs(number))
    random_list1 = []
    for i in range(number):
        random_list1.append(random.randint(start, end))
    return random_list1

In [34]:
# str_start = random_list(0,4,5)
str_start = [4,3,4,2,1]

In [32]:
def attack_number(str, num, attact_num=0):
    for i in range(num):
        for j in range(i+1,num):
            if str[i]==str[j]:
                attact_num=attact_num + 1
            else:
                if abs(i-j)==abs(str[i]-str[j]):
                    attact_num = attact_num + 1
    return attact_num

In [35]:
attack_number(str_start,5)

4

In [36]:
def find_neighbour(str,num):
    neighbour=[]
    k = 0
    for i in range(num):
        for j in range(num):
            if (j+1) != str[i]:
                neighbour.append(list(str))
                neighbour[k][i]=j+1
                neighbour[k][-1]=(neighbour[k][-1] + 10 + (str[i]-j-1)*(str[i]-j-1))
                k = k+1
    return neighbour


In [50]:
def hill_climbing(str,num):
    frontier = queue.PriorityQueue()
    frontier.put((0,str))
    frontier_attack = 10000
    current_attack = 0
    nodes = 0
    while True:
        current1 = frontier.get()
        current = current1[-1]
        frontier.queue.clear()
        current_attack = attack_number(current,num)
        nodes = 20+nodes
        if current_attack == 0:
            result = current #111
            break
        if frontier_attack <= current_attack:
            result=frontier_queen
            break
        frontier_queen=list(current)
        frontier_attack = current_attack
        neighbour = find_neighbour(current,num)
        for next in neighbour:
            priority = 10*attack_number(next,num)+ 100 + next[num]
            frontier.put((priority,next))
    return result, nodes

def restart(str,num):
    restart_number=-1
    result = []
    start = timeit.default_timer()
    all_nodes=0
    while True:
        peak, thistime_nodes = hill_climbing(str, num)
        all_nodes=all_nodes+thistime_nodes
        result.append(list(peak))
        restart_number=restart_number+1
        end = timeit.default_timer()
        a = attack_number(peak,num)
        if (end-start) > 10 or a == 0:                     # change the range of time by rewriting the 10
            break
        str = random_list(1,num,num)
        str.append(0)
    during_time = end-start
    return result,restart_number,during_time,all_nodes

In [54]:
queen=random_list(1,5,5)
queen.append(0)
queen

[3, 1, 1, 2, 1, 0]

In [55]:
str = [4,3,4,2,1,0]

In [59]:
result, nodes = hill_climbing([4,3,4,2,1,0],5)
print(result)
print(nodes)

[4, 3, 5, 2, 1, 11]
60


In [60]:
result,restart_number,during_time,all_nodes=restart(str,5)

In [61]:
print(result)
print(restart_number)
print(during_time)
print(all_nodes)

[[4, 3, 5, 2, 1, 11], [5, 4, 1, 3, 2, 11], [2, 4, 1, 3, 5, 11]]
2
0.004978162993211299
160


In [215]:
import random
import copy
import queue
import numpy as np
import timeit
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

In [216]:
class Map:
    def __init__(self,industrial, commercial, residential):
        self.industrial = int(industrial)
        self.commercial = int(commercial)
        self.residential = int(residential)
        self.siteAmount = int(industrial) + int(commercial) + int(residential)

    def setToxicSite(self,cooridnates = None):
        if cooridnates is None:
            cooridnates =[]
        self.toxicCoord = copy.deepcopy(cooridnates)

    def setScenicView(self,sceCoord = None):
        if sceCoord is None:
            sceCoord = []
        self.scenicCoord = copy.deepcopy(sceCoord)

    def setCostMap(self,costMap = None):
        if costMap is None:
            costMap = []
        self.costMap = copy.deepcopy(costMap)

    def setParentNode(self,ParentNode):
        self.ParentNode = ParentNode

    def setScore(self,Score):
        self.score = Score

    def MapSize(self,iniMap):
        self.row = len(iniMap)-3
        self.column = len(iniMap[3])

In [241]:
def ReadFromFile(filename):
    with open(filename) as f:
        # whenever \n is encountered splitlines will break and take the encountered part aas a string
        my_list = f.read().splitlines()

    # x is the total content reading from file
    x= []
    # the site number, [0] - industrial, [1] - commercial [2] - residential
    site_number = []
    # scenic view coordinates
    scenicPosition = []
    # toxic view coordinates
    toxicPosition = []

    for line in my_list[3:]:
        x.append(line.split(','))

    for i in range(0,3):
        temp = int(my_list[i])
        site_number.append(temp)
    print(x)
    cost_map = []
    for j in range(0,len(x)):
        line = []
        for m in range(len(x[j])):
            if x[j][m] == 'X':
                cost = 99
                line.append(cost)
                toxicPosition.append([j,m])
            elif x[j][m] == 'S':
                cost = -1
                line.append(cost)
                scenicPosition.append([j,m])
            elif x[j][m] != ',':
                cost = int(x[j][m])
                line.append(cost)
        cost_map.append(line)
    print(cost_map)
    map2 = Map(site_number[0], site_number[1], site_number[2])
    map2.setToxicSite(toxicPosition)
    map2.setScenicView(scenicPosition)
    map2.setCostMap(cost_map)

    return map2

In [262]:
def iniPosition(map):
    position = []
    available_position = []
    num_indus = map.industrial
    num_comme = map.commercial
    num_resid = map.residential

    for i in range(len(map.costMap)):
        for j in range(len(map.costMap[i])):
            if map.costMap[i][j] == -1 or map.costMap[i][j] == 99:
                continue
            else:
                available_position.append((i,j))
    print(available_position)

    # choose the positions
    random_position = random.sample(range(0,len(available_position)),map.siteAmount)

    # get the coordinate
    for n in range(len(random_position)):
        position.append([available_position[random_position[n]][0],available_position[random_position[n]][1],0])
    print(position)
    #1 for industrial, 2 for commercial, 3 for residential
    for n in range(num_indus):
        position[n][2] = 1
    for n in range(num_comme):
        position[n+num_indus][2] = 2
    for n in range(num_resid):
        position[n+num_indus+num_comme][2] = 3

    return position

In [263]:
def ManhattanDistance(a,b):
#     两个坐标
    distance = abs(a[0]-b[0]) + abs(b[1]-a[1])
    return distance

In [264]:
def calculateScore(map,position):
    score = 0

    #pentaly for within 2 tiles from toxic site

    for ele in position:
        for ele_1 in map.toxicCoord:
            if ManhattanDistance(ele[:-1],ele_1) <=2:
                if ele[2] == 1:
                    score = score - 10
                elif ele[2] == 2:
                    score = score - 20
                elif ele[2] == 3:
                    score = score - 20

    #Bonus for near Scenic Site, Residental only
    for ele in position:
        if ele[2] == 3:
            for ele_1 in map.scenicCoord:
                if ManhattanDistance(ele[:-1],ele_1) <= 2:
                    score = score + 10

    # Construction cost on different sites:
    for ele in position:
        cost = map.costMap[ele[0]-1][ele[1]-1]
        score = score - cost

    # I,C,R sites points calculation
    ### First Record the index:
    indus_index = []
    comme_index = []
    resid_index = []

    for l in range(len(position)):
        if position[l][2] == 1:
            indus_index.append(l)
        elif position[l][2] == 2:
            comme_index.append(l)
        elif position[l][2] == 3:
            resid_index.append(l)

    ## industrial site benefit from other industrial sites, 2 tiles 3 points bonus:
    for in_l in range(0,len(indus_index)-1):
        for in_1_2 in range(in_l + 1, len(indus_index)):
            if ManhattanDistance(position[indus_index[in_l]][:-1],position[indus_index[in_1_2]][:-1]) <= 2:
                score = score + 3

    ## Commercial benefit from residential, <= 3, +5:
    for com_l in range(len(comme_index)):
        for res_l in range(len(resid_index)):
            if ManhattanDistance(position[comme_index[com_l]][:-1],position[resid_index[res_l]][:-1]) <= 3:
                score = score + 5

    ## Commercial compete with each other, <=2 , -5
    for com_l_1 in range(0,len(comme_index)-1):
        for com_l_2 in range(com_l_1+1,len(comme_index)):
            if ManhattanDistance(position[comme_index[com_l_1]][:-1],position[comme_index[com_l_2]][:-1]) <=2:
                score = score - 5


    ## Residential near industrial, penalty, <=3, -5
    for res_l_1 in range(len(resid_index)):
        for ind_l_1 in range(len(indus_index)):
            if ManhattanDistance(position[resid_index[res_l_1]][:-1],position[indus_index[ind_l_1]][:-1]) <= 3:
                score = score - 5

    return score

In [265]:
calculateScore(map2,position)

-125

In [266]:
position = iniPosition(map2)
position

[(0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]
[[2, 3, 0], [0, 3, 0], [0, 1, 0]]


[[2, 3, 1], [0, 3, 2], [0, 1, 3]]

In [247]:
random.sample(range(0,10),map2.siteAmount)

[3, 9, 5]

In [242]:
map2 = ReadFromFile('Urban 1.txt')
map2.industrial

[['X', '1', '2', '4'], ['3', '4', 'S', '3'], ['6', '0', '2', '3']]
[[99, 1, 2, 4], [3, 4, -1, 3], [6, 0, 2, 3]]


1

In [243]:
map2.toxicCoord

[[0, 0]]

In [221]:
map2.costMap

[]

In [ ]:
def printMap(result,map):

    row = len(map.costMap)
    column = len(map.costMap[0])
    ind = 0
    com = map.industrial
    res = map.commercial + map.industrial
    site_number = map.commercial + map.industrial + map.residential

    pb = [['O'] * column for i in range(row)]

    for i in range(len(map.toxicCoord)):
        pb[map.toxicCoord[i][0]-1][map.toxicCoord[i][1]-1] = 'X'

    for j in range(len(map.scenicCoord)):
        pb[map.scenicCoord[j][0]-1][map.scenicCoord[j][1]-1] = 'S'

    for k in range(ind,com):
        pb[result[k][0]-1][result[k][1]-1] = 'I'

    for l in range(com,res):
        pb[result[l][0] - 1][result[l][1] - 1] = 'C'

    for m in range(res,site_number):
        pb[result[m][0] - 1][result[m][1] - 1] = 'R'

    for i in range(row):
        if i != 0:
            print("")
        for j in range(column):
            print(str(pb[i][j]), end="")
    print("")